# **Python for data science and data analysis from scratch**


Para esta sesión utilizaremos las siguientes librerías:

**numpy**

**scikit-learn**


---

**imutils**

Librería de código abierto con operaciones básicas para manipulación de imágenes: [imutils](https://www.pyimagesearch.com/2015/02/02/just-open-sourced-personal-imutils-package-series-opencv-convenience-functions/)


---


**opencv**

Librería especializada para visión por computadora en tiempo real, soporta C++, Java y Python


Para esta sesión vamos a buscar anomalías en imágenes.

Una anomalía se define como un evento que se desvía de lo estándar, que pasa raramente o que no sigue un patrón.


Ejemplos de anomalías incluyen:

* Picos marcados en la bolsa de valores debido a eventos mundiales.
* Productos defectuosos en una línea de ensamble.
* Muestras contaminadas en un laboratorio.

En una gráfica de distribución normal, las anomalías existen en los extremos.

![bell-curve](https://analyticstraining.com/wp-content/uploads/2013/01/bell-curve.png)


Estos eventos van a ocurrir, pero en una proporción mucho menor.

Para los algoritmos de aprendizaje automático esto es considerado un problema difícil, porque tendremos muchos ejemplos de eventos normales y pocos anormales.


## Algoritmos de detección de anomalías

![anomaly](https://pyimagesearch.com/wp-content/uploads/2020/01/intro_anomaly_detection_plot.png)

La detección de anomalías puede ser dividida en dos subclases principales:


*   **Detección de outliers o valores extremos**: El dataset de entrada contiene ejemplos de eventos normales y anormales. Estos algoritmos buscan ajustar regiones de los datos de entrenamiento donde los eventos estándar están más concentrados, sin tener en cuenta o aislando los eventos de anomalías. Dichos algoritmos a menudo se entrenan de manera no supervisada (es decir, sin etiquetas). A veces utilizamos estos métodos para ayudar a limpiar y pre-procesar conjuntos de datos antes de aplicar técnicas adicionales de aprendizaje automático.

*   **Detección de novedad**: A diferencia de la detección de valores atípicos, que incluye ejemplos de eventos estándar y de anomalía, los algoritmos de detección de novedad solo tienen los puntos de datos de eventos estándar (es decir, sin eventos de anomalías) durante el tiempo de entrenamiento. Durante el entrenamiento, proporcionamos a estos algoritmos ejemplos etiquetados de eventos estándar (aprendizaje supervisado). En el momento de la prueba / predicción, los algoritmos de detección de novedad deben detectar cuando un punto de datos de entrada es atípico.



## Bosques de aislamiento para detección de anomalías

En esta sesión usaremos el algoritmo de Isolation Forests, el cual es un tipo de algoritmo de conjunto y consiste en múltiples árboles de decisión utilizados para dividir el conjunto de datos de entrada en distintos grupos de elementos internos.

Como se muestra en la figura, los Bosques de Aislamiento aceptan un conjunto de datos de entrada (puntos blancos) y luego construyen una variedad que los rodea.

![isolation forest](https://pyimagesearch.com/wp-content/uploads/2020/01/intro_anomaly_detection_isolation_forest.png)

En el momento de la prueba, el bosque de aislamiento puede determinar si los puntos de entrada caen dentro de la mayoría (eventos estándar, puntos verdes) o fuera del área de alta densidad (eventos de anomalías, puntos rojos).

Para más teoría acerca de este algoritmo tenemos el siguiente [artículo](https://medium.com/@keepler.io/isolation-forest-el-algoritmo-estrella-para-detecci%C3%B3n-de-anomal%C3%ADas-416bb5892f10).

## Configurar el entorno de desarrollo de detección de anomalías

Primero, necesitamos bajar el código para nuestra aplicación: [code.zip](https://drive.google.com/file/d/1Iv-mqwfL5bXxhmT4dN32QW3Bg_Khm_AY/view?usp=sharing)

Usando la terminal entramos al directorio y creamos un entorno virtual con el comando: 

```
python3 -m venv env
```
Después activamos nuestro entorno virtual con:


```
source env/bin/activate
```

Ahora, vamos a instalar los paquetes necesarios para la aplicación:



```
pip install numpy
pip install opencv-contrib-python
pip install imutils
pip install scikit-learn
```



Dentro de la carpeta *pyimagesearch* se encuentra el archivo **features.py**. Este script contiene dos funciones responsables de cargar las imágenes de nuestro dataset del disco y calcular el histograma para cada imagen.

![histograma](https://www.researchgate.net/profile/Jose_Antonio_Taquia_Gutierrez/publication/322016396/figure/fig6/AS:668393673867271@1536368878493/Figura-7-Histograma-de-color-Imagen-A.png)

Como en algoritmos anteriores, tendremos una fase de entrenamiento y otra de prueba.

El archivo de **train_anomaly_detector.py** calcula los atributos y entrena el modelo de Isolation Forest para detectar anomalías, el resultado se guarda como anomaly_detector.model.

La siguiente parte es el script de **test_anomaly_detector.py**, el cual lee una imagen y determina si es o no una anomalía.



## Implementación de las funciones auxiliares de extracción de características

Antes de que podamos entrenar un modelo de aprendizaje automático para detectar anomalías y valores atípicos, primero debemos definir un proceso para cuantificar y caracterizar los contenidos de nuestras imágenes de entrada.

Para lograr esta tarea, utilizaremos histogramas de color.

Los histogramas de color son métodos simples pero efectivos para caracterizar la distribución de color de una imagen.



In [0]:
# import the necessary packages
from imutils import paths
import numpy as np
import cv2
def quantify_image(image, bins=(4, 6, 3)):
	# compute a 3D color histogram over the image and normalize it
	hist = cv2.calcHist([image], [0, 1, 2], None, bins,
		[0, 180, 0, 256, 0, 256])
	hist = cv2.normalize(hist, hist).flatten()
	# return the histogram
	return hist

Para aprender más acerca de image processing y computer vision tenemos esta página: [www.pyimagesearch.com](https://www.pyimagesearch.com/practical-python-opencv/).

La siguiente función maneja dos cosas:

* Aceptar la ruta a un directorio que contiene el dataset de imágenes.

* Recorre las imágenes del directorio y llama el método de quantify_image para procesarlas


In [0]:
def load_dataset(datasetPath, bins):
	# grab the paths to all images in our dataset directory, then
	# initialize our lists of images
	imagePaths = list(paths.list_images(datasetPath))
	data = []
	# loop over the image paths
	for imagePath in imagePaths:
		# load the image and convert it to the HSV color space
		image = cv2.imread(imagePath)
		image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
		# quantify the image and update the data list
		features = quantify_image(image, bins)
		data.append(features)
	# return our data list as a NumPy array
	return np.array(data)

## Implementar el script de entrenamiento de detección de anomalías con scikit-learn

Con nuestras funciones de ayuda implementadas, ahora podemos pasar a entrenar un modelo de detección de anomalías.

Como se mencionó anteriormente en esta sesión, utilizaremos un bosque de aislamiento para ayudar a determinar los puntos de datos de anomalías / novedades.

Nuestra implementación de Bosques de aislamiento proviene de la biblioteca [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html).


In [0]:
# import the necessary packages
from pyimagesearch.features import load_dataset
from sklearn.ensemble import IsolationForest
import argparse
import pickle
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True,
	help="path to dataset of images")
ap.add_argument("-m", "--model", required=True,
	help="path to output anomaly detection model")
args = vars(ap.parse_args())

Ahora, podemos entrenar nuestro modelo

In [0]:
# load and quantify our image dataset
print("[INFO] preparing dataset...")
data = load_dataset(args["dataset"], bins=(3, 3, 3))
# train the anomaly detection model
print("[INFO] fitting anomaly detection model...")
model = IsolationForest(n_estimators=100, contamination=0.01,
	random_state=42)
model.fit(data)

Después de entrenar nuestro modelo, se guarda en disco:

In [0]:
# serialize the anomaly detection model to disk
f = open(args["model"], "wb")
f.write(pickle.dumps(model))
f.close()

## Entrenar el detector de anomalías

Vamos a correr el siguiente comando desde nuestra terminal:



```
python train_anomaly_detector.py --dataset forest --model anomaly_detector.model
```



## Probar el detector de anomalías

El siguiente script **test_anomaly_detector.py** carga el modelo que creamos en el paso anterior.

Carga, pre-procesa y cuantifica una imagen de prueba.

Hace una predicción para determinar si la imagen es un bosque o no (una anomalía).

Muestra el resultado:



In [0]:
# import the necessary packages
from pyimagesearch.features import quantify_image
import argparse
import pickle
import cv2
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-m", "--model", required=True,
	help="path to trained anomaly detection model")
ap.add_argument("-i", "--image", required=True,
	help="path to input image")
args = vars(ap.parse_args())

In [0]:
# load the anomaly detection model
print("[INFO] loading anomaly detection model...")
model = pickle.loads(open(args["model"], "rb").read())
# load the input image, convert it to the HSV color space, and
# quantify the image in the *same manner* as we did during training
image = cv2.imread(args["image"])
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
features = quantify_image(hsv, bins=(3, 3, 3))

In [0]:
# use the anomaly detector model and extracted features to determine
# if the example image is an anomaly or not
preds = model.predict([features])[0]
label = "anomaly" if preds == -1 else "normal"
color = (0, 0, 255) if preds == -1 else (0, 255, 0)
# draw the predicted label text on the original image
cv2.putText(image, label, (10,  25), cv2.FONT_HERSHEY_SIMPLEX,
	0.7, color, 2)
# display the image
cv2.imshow("Output", image)
cv2.waitKey(0)

Para correr la prueba usamos el comando:



```
python test_anomaly_detector.py --model anomaly_detector.model --image examples/forest_cdmc290.jpg 
```



**Ejercicio**

Probar el algoritmo con otro dataset para ver si es capaz de detectar anomalías. 

Por ejemplo con el siguiente dataset de [hacker rank](https://www.hackerrank.com/challenges/digital-camera-day-or-night/problem).

En qué otros escenarios puede servir nuestro modelo? Detectar animales perros/gatos, ciudad/provincia, etc.?